In [18]:
from pathlib import Path
import pandas as pd
import numpy as np

In [19]:
!pip install openpyxl

In [20]:
# Files

markets_old = Path("markets_101125.csv")   
markets_new = Path("markets_111125.csv")   

In [21]:
KEY_COLS = ["id"]
IGNORE_COLS = []   # z.B. ["updated_at", "last_modified_by"]

CSV_READ_KW = dict(
    sep=";",
    decimal=".",
    encoding="utf-8",
    dtype=None,
    keep_default_na=True,
    na_values=["", "NA", "NaN", "NULL"]
)

In [22]:
def normalize_dataframe(df: pd.DataFrame, key_cols, ignore_cols):
    """
    Vereinheitlicht Spaltenreihenfolge (Keys zuerst),
    entfernt ignorierte Spalten, trimmt Strings.
    """
    df = df.copy()
    # Trim in Stringspalten
    for c in df.select_dtypes(include=["object","string"]).columns:
        df[c] = df[c].astype("string").str.strip()
    # Ignorierte Spalten entfernen
    drop_cols = [c for c in ignore_cols if c in df.columns]
    if drop_cols:
        df = df.drop(columns=drop_cols)
    # Spaltenreihenfolge: Keys zuerst
    rest = [c for c in df.columns if c not in key_cols]
    cols = list(key_cols) + rest
    return df[cols]

In [23]:
def compare_dataframes(df_old, df_new, key_cols):


    # Outer-Join auf Schlüssel, um Added/Removed zu erkennen
    merged_index = pd.merge(
        df_old[key_cols].drop_duplicates(), 
        df_new[key_cols].drop_duplicates(),
        on=key_cols, how="outer", indicator=True
    )
    added_keys = merged_index.loc[merged_index["_merge"] == "right_only", key_cols]
    removed_keys = merged_index.loc[merged_index["_merge"] == "left_only", key_cols]
    common_keys = merged_index.loc[merged_index["_merge"] == "both", key_cols]

    # Added / Removed Zeilen
    added_rows = added_keys.merge(df_new, on=key_cols, how="left")
    removed_rows = removed_keys.merge(df_old, on=key_cols, how="left")

    # Modified Zellen
    if common_keys.empty:
        modified_cells = pd.DataFrame(columns=key_cols + ["column","old","new"])
        return added_rows, removed_rows, modified_cells

    old_common = common_keys.merge(df_old, on=key_cols, how="left")
    new_common = common_keys.merge(df_new, on=key_cols, how="left")

    # Spalten vergleichen (alle außer Keys)
    value_cols = [c for c in df_new.columns if c not in key_cols]
    value_cols = [c for c in value_cols if c in old_common.columns]

    diffs = []
    for col in value_cols:
        a = old_common[col]
        b = new_common[col]
        neq = ~((a == b) | (a.isna() & b.isna()))
        if neq.any():
            diff_part = pd.concat([
                common_keys.reset_index(drop=True), 
                pd.Series([col]*len(common_keys), name="column"),
                a.reset_index(drop=True).rename("old"),
                b.reset_index(drop=True).rename("new")
            ], axis=1)
            modified = diff_part[neq.values]
            diffs.append(modified)

    modified_cells = pd.concat(diffs, ignore_index=True) if diffs else pd.DataFrame(columns=key_cols + ["column","old","new"])
    return added_rows, removed_rows, modified_cells

In [24]:
# Daten laden und normalisieren

df_old = pd.read_csv(markets_old, **CSV_READ_KW)
df_new = pd.read_csv(markets_new, **CSV_READ_KW)

# Validierung
missing_keys_old = [k for k in KEY_COLS if k not in df_old.columns]
missing_keys_new = [k for k in KEY_COLS if k not in df_new.columns]
if missing_keys_old or missing_keys_new:
    raise ValueError(f"Fehlende Schlüsselspalten – alt: {missing_keys_old}, neu: {missing_keys_new}")

# Normalisieren
df_old_n = normalize_dataframe(df_old, KEY_COLS, IGNORE_COLS)
df_new_n = normalize_dataframe(df_new, KEY_COLS, IGNORE_COLS)

print("Alt-Shape:", df_old_n.shape, "| Neu-Shape:", df_new_n.shape)
print("Spalten:", list(df_new_n.columns))

Alt-Shape: (44, 32) | Neu-Shape: (44, 32)
Spalten: ['id', 'name', 'shortname', 'strasse', 'bezirk', 'plz_ort', 'von', 'bis', 'oeffnungszeiten', 'Mo', 'Di', 'Mi', 'Do', 'Fr', 'Sa', 'So', 'closed-exc', 'hours-exc', 'w3', 'bemerkungen', 'bemerkungen_en', 'lat', 'lng', 'ignore', 'immer-kostenlos', 'international', 'barrierefrei', 'action', 'short_distance', 'train', 'image', 'urheberschaft']


In [25]:
# Vergleich durchführen

added_rows, removed_rows, modified_cells = compare_dataframes(df_old_n, df_new_n, KEY_COLS)

print(f"Neu hinzugekommen: {len(added_rows)} Zeilen")
print(f"Entfernt:          {len(removed_rows)} Zeilen")
print(f"Geänderte Zellen:  {len(modified_cells)} Zellen")

display(added_rows.head(10))
display(removed_rows.head(10))
display(modified_cells.head(20))


Neu hinzugekommen: 0 Zeilen
Entfernt:          0 Zeilen
Geänderte Zellen:  0 Zellen


,id,name,shortname,strasse,bezirk,plz_ort,von,bis,oeffnungszeiten,Mo,...,lng,ignore,immer-kostenlos,international,barrierefrei,action,short_distance,train,image,urheberschaft


,id,name,shortname,strasse,bezirk,plz_ort,von,bis,oeffnungszeiten,Mo,...,lng,ignore,immer-kostenlos,international,barrierefrei,action,short_distance,train,image,urheberschaft


,id,column,old,new


In [26]:
# Übersicht pro Schlüssel 

if not modified_cells.empty:
    pivot = (
        modified_cells
        .assign(changed=1)
        .pivot_table(index=KEY_COLS, columns="column", values="changed", aggfunc="sum", fill_value=0)
        .reset_index()
    )
    display(pivot.head(20))
else:
    print("Keine geänderten Zellen.")


Keine geänderten Zellen.


In [29]:
# Ergebnisse speichern
from datetime import datetime
stamp = datetime.now().strftime("%d%m%y")

OUT_DIR = Path(f"vergleich_report_{stamp}")
OUT_DIR.mkdir(exist_ok=True, parents=True)

added_rows.to_csv(OUT_DIR / "added_rows.csv", index=False, encoding="utf-8")
removed_rows.to_csv(OUT_DIR / "removed_rows.csv", index=False, encoding="utf-8")
modified_cells.to_csv(OUT_DIR / "modified_cells.csv", index=False, encoding="utf-8")

# Optional: alles in eine Excel-Datei schreiben
with pd.ExcelWriter(OUT_DIR / "report.xlsx", engine="openpyxl") as xw:
    added_rows.to_excel(xw, sheet_name="Added", index=False)
    removed_rows.to_excel(xw, sheet_name="Removed", index=False)
    modified_cells.to_excel(xw, sheet_name="ModifiedCells", index=False)

print("Ergebnisse gespeichert in:", OUT_DIR.resolve())


Ergebnisse gespeichert in: /Users/norahunger/Documents/GitHub/weihnachtsmarktkarte/data_processing/vergleich_report_111125
